# Data Wrangling
1. Data Collection 
2. Data Definition
3. Data Cleaning

## 1. Data Collection

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The structured dataset originated from the Better Buildings Neighborhood Program comes in two different tables:
- bbnp_installed_measures_res_sf: with information about retrofit measures installed in each project
- bbnp_retrofit_project_table_res_sf: architecture and construction attributes of each project

In this section, I will blend both tables to invertigate relationships between them.  

Moreover, I will combine the dataset from Better Buildings Neighborhood Program with climate data from the National Oceanic and Atmospheric Administration (NOAA); namely, annual average cooling degree days and annual average heating degree days, which represent the most important external factors to determine the amount of heating and cooling in a building. The link between project and climate data will be location-wise (state).

In [2]:
# Import dataset from Better Buildings Neighborhood Program 
measures_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/bbnp_installed_measures_res_sf.csv'
project_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/bbnp_retrofit_project_table_res_sf.csv'

# Load them into pandas dataframes
measures_df = pd.read_csv(measures_file, index_col=None, low_memory=False)
project_df = pd.read_csv(project_file, index_col=None, low_memory=False)

# Import climatic data from National Oceanic and Atmospheric Administration (NOAA) and location:
cool_degree_days_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/ann-cldd-normal.txt'
heat_degree_days_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/ann-htdd-normal.txt'
station_location_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/zipcodes-normals-stations.txt'
zip_codes_file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/raw/zip_code_database.csv'

# Load them into pandas dataframes
cool_degree_days_df = pd.read_csv(cool_degree_days_file, sep="        ", engine='python', names=['STATION_ID', 'COOLING_DD'], index_col=None)
heat_degree_days_df = pd.read_csv(heat_degree_days_file, sep="        ", engine='python', names=['STATION_ID', 'HEATING_DD'], index_col=None)
station_location_df = pd.read_csv(station_location_file, sep=" ", usecols=[0, 1], names=['STATION_ID', 'ZIP_CODE'], index_col=None)
zip_codes_df = pd.read_csv(zip_codes_file, usecols=['zip', 'state'], index_col=None)

### First, I will consolidate the climate dataset

In [3]:
# Standarize character length and data type in station_location_df

station_location_df['ZIP_CODE'] = station_location_df['ZIP_CODE'].apply(lambda x: '{0:0>5}'.format(x)).astype('int64') 
station_location_df['STATION_ID'] = station_location_df['STATION_ID'].astype('string')

In [4]:
# Standarize character length, data type and column names in zip_codes_df

zip_codes_df['zip'] = zip_codes_df['zip'].apply(lambda x: '{0:0>5}'.format(x)).astype('int64') 
zip_codes_df['state'] = zip_codes_df['state'].astype('string')
zip_codes_df.columns = ['ZIP_CODE', 'STATE']

In [5]:
# Standarize data type and removing forward blank spaces in climate data

cool_degree_days_df = cool_degree_days_df.astype('string')
heat_degree_days_df = heat_degree_days_df.astype('string')

cool_degree_days_df['COOLING_DD'] = cool_degree_days_df['COOLING_DD'].str.strip()
heat_degree_days_df['HEATING_DD'] = heat_degree_days_df['HEATING_DD'].str.strip()

In [6]:
# Check null value amount

cool_degree_days_df.info(), heat_degree_days_df.info(), station_location_df.info(), zip_codes_df.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATION_ID  7501 non-null   string
 1   COOLING_DD  7471 non-null   string
dtypes: string(2)
memory usage: 117.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATION_ID  7501 non-null   string
 1   HEATING_DD  7237 non-null   string
dtypes: string(2)
memory usage: 117.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9794 entries, 0 to 9793
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATION_ID  9794 non-null   string
 1   ZIP_CODE    9794 non-null   int64 
dtypes: int64(1), string(1)
memory usage: 153.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42522 entries, 0 to

(None, None, None, None)

In [7]:
# Merge heating degree days into cooling degree days

climate_df = pd.merge(cool_degree_days_df, heat_degree_days_df, how='inner', on='STATION_ID')

In [8]:
# Add station state location to climate data

climate_df = climate_df.merge(station_location_df, how='left', on='STATION_ID')

In [9]:
# Add zip codes to climate data

climate_df = climate_df.merge(zip_codes_df, how='left', on='ZIP_CODE') 

In [10]:
# Remove all null entries (Given the limited amount of null observations) 

climate_df.dropna(axis=0, inplace=True)

For every data entry in COOLING_DD and HEATING_DD there is ending character as a completeness flag:

- C = complete (all 30 years used)
- S = standard (no more than 5 years missing and no more than 3 consecutive years missing among the sufficiently complete years)
- R = representative (observed record utilized incomplete, but value was scaled or based on filled values to be representative of the full period of record)
- P = provisional (at least 10 years used, but not sufficiently complete to be labeled as standard or representative)
- Q = quasi-normal (at least 2 years per month, but not sufficiently complete to be labeled as provisional or any other higher flag code
- Blank = the data value is reported as a special value such as 9999

I will keep C, S, and R observations.

In [11]:
# Check % of C, S and R flags in cooling degree days

cc = round(climate_df.COOLING_DD.str.count('C').sum() / climate_df.shape[0], 2)
cs = round(climate_df.COOLING_DD.str.count('S').sum() / climate_df.shape[0], 2) 
cr = round(climate_df.COOLING_DD.str.count('R').sum() / climate_df.shape[0], 2) 

cc, cs, cr

(0.03, 0.34, 0.48)

In [12]:
# Check % of C, S and R flags in heating degree days

hc = round(climate_df.HEATING_DD.str.count('C').sum() / climate_df.shape[0], 2)
hs = round(climate_df.HEATING_DD.str.count('S').sum() / climate_df.shape[0], 2) 
hr = round(climate_df.HEATING_DD.str.count('R').sum() / climate_df.shape[0], 2) 

hc, hs, hr

(0.03, 0.34, 0.48)

85% of data have any of those flags, so I can safely remove the rest unreliable data without loosing too many observations

In [13]:
# Keep entries ending with C, S or R

climate_df = climate_df[climate_df.COOLING_DD.str.endswith('C') |
                        climate_df.COOLING_DD.str.endswith('S') |
                        climate_df.COOLING_DD.str.endswith('R')]

In [14]:
# Keep entries ending with C, S or R

climate_df = climate_df[climate_df.HEATING_DD.str.endswith('C') |
                        climate_df.HEATING_DD.str.endswith('S') |
                        climate_df.HEATING_DD.str.endswith('R')]

In [15]:
# Create a funtion to remove last character 
def remove_last_char(series):
    '''Removes last character from each entry in a Series with all string dtypes'''
    for ix, row in series.iteritems():
        series[ix] = row[:-1]

In [16]:
# Remove last character in COOLING_DD and HEATING_DD data

remove_last_char(climate_df['COOLING_DD'])
remove_last_char(climate_df['HEATING_DD'])

In [17]:
# Check for df formatting

climate_df.head()

,STATION_ID,COOLING_DD,HEATING_DD,ZIP_CODE,STATE
38,RQC00666992,492,380,718.0,PR
39,RQC00667292,5398,0,795.0,PR
40,RQC00668306,5117,0,925.0,PR
41,RQC00668881,4871,0,685.0,PR
42,RQC00669521,5096,0,976.0,PR


In [18]:
# Drop station id and zip code info. No longer need them

climate_df.drop(['STATION_ID', 'ZIP_CODE'], axis=1, inplace=True)

In [19]:
climate_df['COOLING_DD'] = pd.to_numeric(climate_df['COOLING_DD'])
climate_df['HEATING_DD'] = pd.to_numeric(climate_df['HEATING_DD'])



In [20]:
# Group by state to generate a link column with the projects information table 

climate_by_st = climate_df.groupby(by='STATE').mean()

In [21]:
climate_by_st

,COOLING_DD,HEATING_DD
STATE,,
AK,5.111111,8397.138889
AL,1954.056180,2657.449438
AR,1829.100000,3411.490000
AZ,2143.371795,3182.641026
CA,1105.555874,3156.432665
CO,509.427807,6867.032086
CT,596.791667,6203.041667
DC,1497.333333,4202.000000
DE,1192.666667,4551.888889


### Now, I will consolidate building attributes and project attributes

In [22]:
# Inspecting dfs

measures_df.shape, project_df.shape

((75110, 89), (75110, 31))

In [23]:
measures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75110 entries, 0 to 75109
Data columns (total 89 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   AWARDEENUMBER                75110 non-null  int64  
 1   PROJECTID                    75110 non-null  int64  
 2   INST_FURNACE                 75110 non-null  int64  
 3   INST_FURNACE_AFUE            3478 non-null   float64
 4   INST_FURNACE_FUEL            3629 non-null   object 
 5   INST_BOILER                  75110 non-null  int64  
 6   INST_BOILER_AFUE             963 non-null    float64
 7   INST_BOILER_FUEL             1062 non-null   object 
 8   INST_WOOD_STOVE              75110 non-null  int64  
 9   INST_WATER_HEATER            75110 non-null  int64  
 10  INST_WATER_HEATER_FUEL       1835 non-null   object 
 11  INST_WATER_HEATER_EF         976 non-null    float64
 12  INST_HEAT_PUMP               75110 non-null  int64  
 13  INST_HEAT_PUMP_T

In [24]:
project_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75110 entries, 0 to 75109
Data columns (total 31 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   AWARDEENUMBER                           75110 non-null  int64  
 1   PROJECTID                               75110 non-null  int64  
 2   RETROFITJOBHOURS                        63670 non-null  float64
 3   RETROFITINVOICEDCOST                    66281 non-null  float64
 4   BUILDINGACTIVITYTYPE                    71212 non-null  float64
 5   OCCUPANCY                               54373 non-null  float64
 6   ISLOWINCOME                             32566 non-null  object 
 7   FLOORAREA                               75110 non-null  object 
 8   FLOORAREAUNIT                           58863 non-null  object 
 9   ISDIRECTINSTALLATIONUSED                56195 non-null  object 
 10  LOANAMOUNT                              15118 non-null  fl

In [25]:
# Check for uniqueness ID for projects.

measures_df['PROJECTID'].nunique(), project_df['PROJECTID'].nunique()

(75110, 75110)

In [26]:
# Consolidate all tables

df = pd.merge(measures_df, project_df, how='inner', on='PROJECTID')
df.drop(['AWARDEENUMBER_x', 'AWARDEENUMBER_y', 'PROJECTID'], axis=1, inplace=True)

df = pd.merge(df, climate_by_st, how='left', on='STATE')

**There are many variables that are not useful for our analyisis at this point:**
1. Irrelevant project attributes.
2. Undefined retrofit measures.
3. Flags about building conditioning systems and construction elements removed.

In [27]:
# Remove other columns with little or not interesting information (case 1 and 2)

columns2drop = ['STATE_BASED_ON', 
                'FLOORAREAUNIT', 
                'RETROFITSTARTMONTHYEAR', 
                'RETROFITCOMPLETIONMONTHYEAR', 
                'AUDITCOMPLETIONMONTHYEAR',
                'LOANAPPROVALMONTHYEAR',
                'INST_CORE_ENERGY',
                'INST_HEALTH_SAFETY',
                'REP_WINDOWS',
                'LOANAMOUNT',
                'DAYSFROMRETROFITSTARTTOCOMPLETION',
                'DAYSFROMAUDITCOMPLETIONTORETROFITSTART']

df.drop(columns2drop, axis=1, inplace=True)

In [28]:
# Remove RMV columns (case 3).

columns2drop = [i for i in df.columns if i.startswith('RMV')]
df.drop(columns2drop, axis=1, inplace=True)

In [29]:
df.shape

(75110, 90)

In [30]:
# Save consolidated data

file = 'C:/Users/javie/OneDrive/Documents/springboard_projects/Capstone-2/data/interim/consolidated.csv'
df.to_csv(file)